In [1]:
import numpy as np
from scipy.stats import beta, norm
from scipy.optimize import minimize
np.random.seed(666)

In [2]:
# Constants
COST = 12000
CAPACITY = 150
COMPENSATION_FACTOR = 2.5
REFUND_FACTOR = 0.5
DEMAND_INTERCEPT = 600
DEMAND_COEFFICIENT = -5
DEMAND_STD = 10
NOSHOW_MEAN = 0.1
NOSHOW_STD = 0.025
NUM_REALIZATIONS = 1000

In [3]:
# Convert mean and standard deviation to a and b parameters for the beta distribution
NOSHOW_A = ((1 - NOSHOW_MEAN) / NOSHOW_STD**2 - 1 / NOSHOW_MEAN) * NOSHOW_MEAN**2
NOSHOW_B = NOSHOW_A * (1 / NOSHOW_MEAN - 1)

# Random realizations
realization_forecast_error = np.random.normal(0, DEMAND_STD, NUM_REALIZATIONS)
realization_no_shows_fraction = beta.rvs(NOSHOW_A, NOSHOW_B, size=NUM_REALIZATIONS)

In [4]:
# Define the mean profit function
def mean_profit(price, reservations):
    profits = []
    for i in range(NUM_REALIZATIONS):
        # Calculate demand
        demand = DEMAND_INTERCEPT + DEMAND_COEFFICIENT * price + realization_forecast_error[i]
        # Calculate the number of tickets sold
        sales = np.minimum(reservations, demand)
        # Calculate the no-shows
        no_shows = sales * realization_no_shows_fraction[i]
        # Calculate the passengers that show up
        passengers = sales - no_shows
        # If we have overbooked, we need to compensate some passengers
        if passengers > CAPACITY:
            compensation = (passengers - CAPACITY) * price * COMPENSATION_FACTOR
        else:
            compensation = 0
        # We refund no-shows
        refund = no_shows * price * REFUND_FACTOR
        # Calculate the total revenue from selling tickets
        revenue = sales * price
        # Calculate the total cost
        cost = COST + compensation + refund
        # Profit is revenue minus cost
        profits.append(revenue - cost)
    # Return the average profit across all realizations
    return np.mean(profits)

# The objective function to minimize (we add a minus sign to turn the maximization problem into a minimization problem)
def objective(x):
    return -mean_profit(x[0], x[1])

# Initial guess
x0 = [100, 150]

# Bounds (prices and reservations can't be negative)
bounds = [(0, None), (0, None)]

# Minimize the objective function
res = minimize(objective, x0, bounds=bounds)

In [5]:
print(f"Optimal price: ${round(res.x[0])}")
print(f"Optimal number of reservations: {round(res.x[1])}")

Optimal price: $87
Optimal number of reservations: 165


In [6]:
# Define the profit function with fixed reservations
def profit_fixed_reservations(price):
    # reservations are fixed at capacity
    reservations = CAPACITY
    return mean_profit(price, reservations)

# The objective function to minimize with fixed reservations
def objective_fixed_reservations(x):
    return -profit_fixed_reservations(x)

# Minimize the objective function with fixed reservations
res_fixed_reservations = minimize(objective_fixed_reservations, x0[0])

In [7]:
print(f"Optimal price with fixed reservations: ${round(res_fixed_reservations.x[0])}")

Optimal price with fixed reservations: $89


In [9]:
round(mean_profit(87, 166))

1175

In [10]:
mean_profit(87, 150)

374.9016698367462

In [11]:
mean_profit(89, 150)

515.5594069239061